In [82]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from skimage.morphology import remove_small_holes, area_opening, skeletonize, area_closing, binary_erosion, binary_dilation
import pandas as pd
from matplotlib import rcParams
from skimage.measure import find_contours


In [2]:
COLOR = 'black'
plt.style.use("default")
rcParams["font.family"] = "serif"
rcParams["mathtext.fontset"] = "cm"
rcParams['figure.figsize'] = (28, 18)
rcParams['figure.titlesize'] = 20
rcParams['figure.dpi'] = 50
rcParams['axes.grid'] = True
rcParams['axes.facecolor'] = 'lightgray'
rcParams['legend.fontsize'] = 25
rcParams['axes.titlesize'] = 60
rcParams['axes.labelsize'] = 50
rcParams['ytick.labelsize'] = 20
rcParams['xtick.labelsize'] = 20
rcParams['text.color'] = COLOR
rcParams['text.color'] = COLOR
rcParams['axes.labelcolor'] = COLOR
rcParams['xtick.color'] = COLOR
rcParams['ytick.color'] = COLOR
rcParams['grid.color'] = COLOR
rcParams['text.latex.preamble'] = "\n".join([
    r'\usepackage[version=3]{mhchem}'
])

In [3]:
THRESHOLD = 120
MAX_PIXEL_VALUE = 255

### funciones

In [100]:

def cortar(frame, limites, PRUEBA=False, nombre='CORTE'):
    min_x, max_x, min_y, max_y = limites
    corte = frame[min_y:max_y, min_x:max_x]
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', corte)
    return corte

def gris(frame, PRUEBA=False, nombre='GRAYSCALE'):
    im_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', im_gray)
    return im_gray

def binarize(frame, th=THRESHOLD, PRUEBA=False, nombre='BINARY'):
    _, im_binary = cv.threshold(frame, th, MAX_PIXEL_VALUE, cv.THRESH_BINARY)
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', im_binary)
    return im_binary

def guillotina(frame, PRUEBA=False, nombre='DECAPITADO'):
    y, x = np.where(frame<=130)
    frame[y, x] = np.mean(frame)
    frame -= np.min(frame)
    frame = (frame/np.max(frame))*255
    
    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', frame)
    return frame

def binarize_TAIL(frame, PRUEBA=False, nombre='BINARY_TAIL'):
    _, im_binary = cv.threshold(frame, THRESHOLD-15, MAX_PIXEL_VALUE, cv.THRESH_BINARY)
    arr = im_binary > 0
    im_binary = remove_small_holes(arr)*255

    # ESTO DE ACÁ COMENTADO ES LO QUE ACELERABA EL AREA OPENING:
    # no es el código más limpio del mundo pero funca, ahora no hacemos pero no lo borro ni corrijo todavía pq estoy trabajando cosas con eso
    y, x = np.where(im_binary==0)
    im_bin = im_binary[min(y)-1:max(y)+2, min(x)-1:max(x)+2]
    im_bin = area_opening(im_bin, 200)
    im_binary[min(y)-1:max(y)+2, min(x)-1:max(x)+2] = im_bin

    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', im_binary)
    return im_binary

def skeleton(frame, PRUEBA=False, nombre='ESQUELETO'):
    arr = frame < 255
    sk = skeletonize(arr)*255
    _, x = np.where(sk!=0)

    # esto cierra los globitos, es lo que faltaba antes:
    for c in np.unique(x):
        r = np.where(sk[:, c]!=0)[0]
        for p in r:
            sk[p, c] = 0
        sk[int(np.mean(r)), c] = 255
    y, x = np.where(sk!=0)

    if PRUEBA:
        cv.imwrite(f'Imaging tests\\{nombre}.jpg', sk)
    return sk, x, y

def track_TAIL(vs, limites, SHOW=False):
        # coordenadas de la cola en cada frame
        tail_x = np.array([])
        tail_y = np.array([])        
        frame_count = vs.get(7)
        tiempo = [f for f in range(1, int(frame_count))]
        while(vs.isOpened()):
            ret, frame = vs.read()
            if ret:
                frame = cortar(frame, limites)

                # para que sólo tome cuando la cola está completa:
                im_izq = frame[:, 0:1]
                im_izq = cv.bitwise_not(binarize(gris(im_izq), th=170))
                im_der = frame[:, -2:-1]
                im_der = cv.bitwise_not(binarize(gris(im_der), th=170))
                if np.sum(im_izq) == 0:
                    if np.sum(im_der) == 0:
                        frame_bn = binarize_TAIL(guillotina(gris(frame)))
                        _, x, y = skeleton(frame_bn)
                        tail_x = np.append(tail_x, x)
                        tail_y = np.append(tail_y, y)
                        if SHOW:
                            frame[y, x] = [0, 0, 255]
                            cv.imshow('frame', frame)

                        if cv.waitKey(1) & 0xFF == ord('q'):
                            break
                else:
                    break
            else:
                print('Video no abierto')
                break
        cv.destroyAllWindows()

        return np.array(tiempo), tail_x, tail_y

In [8]:
# lim = [319, 950, 250, 505]
lim = [319, 1000, 250, 505]

In [27]:
im = cv.imread('Capturas\\Camo Snapshot 2024-05-17 12-04-24 - 10Hz.jpg')
im = cortar(im, lim)


In [101]:
video = cv.VideoCapture('ECO20-45mm-PECERA1\\ECO20-45mm-1,49Vpp-8Hz.mp4')
print(video.isOpened())
t = track_TAIL(video, lim, SHOW=True)

# con el blur y el area opening a veces se pierde el final de alguna cola (500mVpp, 3Hz x ej)
# jugar con el orden de los filtros capaz

True


In [94]:
import os

for filename in os.listdir('ECO20-45mm-PECERA1'):
    spl = filename.split('.')[0].split('-')
    v,f = spl[2],spl[3]
    video = cv.VideoCapture('ECO20-45mm-PECERA1/'+filename)
    lim = [319, 950, 250, 505] #[xleft, xright, ytop, ydown]
    t,x,y = track_TAIL(video, lim, SHOW=True)

IndexError: list index out of range